In [38]:
from tikapi import TikAPI, ValidationException, ResponseException
import json
import glob
import os
import re
import pandas as pd
import numpy as np

api = TikAPI("NDW6LmIv1AbBe3EUqilHvsF0Yq8kgpiwbWEiGK2xXBdTj36h")
User = api.user(accountKey="6EjyV4OptbvUz1cKgMTnvbXNH1wKPqLVV42jKlzydbVrFLnb")

_data_path = "./data/"
video_ids_file = _data_path + "video_list.csv"

max_requests_for_each_video = 50
number_of_comment_each_request = 30 #(max possible value is 30)

# Load video IDs

In [6]:
df = pd.read_csv(video_ids_file)
# df = df[df["Influencer/tiktoker"]=="Arianna Biella"]


# extract video ids from links
df["id"] = [re.sub("video/", "", re.findall("video/[0-9]{19}", link)[0]) for link in df["Link"]]
print(f"First 5 video ids:\n{df.head(5)['id']}")

df

First 5 video ids:
0    7144986246215929094
1    7146104387079900421
2    7138834940187151622
3    7140954969871600901
4    7140961766028102918
Name: id, dtype: object


,Influencer/tiktoker,Politician,Date,Subject,Link,Done,id
0,NaN,Meloni,19/09/2022,RDC,https://www.tiktok.com/@giorgiameloni_ufficial...,False,7144986246215929094
1,NaN,Salvini,23/09/2022,Renewable energy,https://www.tiktok.com/@matteosalviniufficiale...,False,7146104387079900421
2,NaN,Berlusconi,02/09/2022,Young generation,https://www.tiktok.com/@silvio.berlusconi/vide...,False,7138834940187151622
3,Arianna Biella,NaN,08/09/2022,Personal preferences on proposed issues,https://www.tiktok.com/@ariannabiellas/video/7...,False,7140954969871600901
4,Arianna Biella,NaN,08/09/2022,Personal preferences on proposed issues,https://www.tiktok.com/@ariannabiellas/video/7...,False,7140961766028102918
...,...,...,...,...,...,...,...
81,NaN,Calenda,17/09/2022,alliance with Renzi,https://www.tiktok.com/@carlocalendaofficial/v...,NaN,7144415435360832773
82,NaN,Calenda,19/09/2022,Calenda vs Di Maio,https://www.tiktok.com/@carlocalendaofficial/v...,NaN,7145138456635755782
83,NaN,Calenda,20/09/2022,PNRR,https://www.tiktok.com/@carlocalendaofficial/v...,NaN,7145536159203020038
84,NaN,Calenda,22/09/2022,Calenda vs Lega,https://www.tiktok.com/@carlocalendaofficial/v...,NaN,7146240836244753670


# Devine functions

In [127]:
def get_comments(id: str, max_requests: int = max_requests_for_each_video) -> dict:
    """ 
    Downloads comments of TikTok videos using TikAPI. 
    """
    j = 0                       # single video request counter
    total_n_comments_read = 0   # total comments counter
    comments = []               # list to collect comments

    # try to request comments for video id
    try:
        response = User.posts.comments.list(
            media_id = id,
            count = number_of_comment_each_request # (max possible value is 30)
        )
    
        response_json = response.json()
        
        # set request counter and loop over responses
        j=1
        while(response):    
            
            response_json = response.json()
            
            if response == None:
                break # no more responses

            if response_json['comments'] == None:
                print("Continue!")
                # response = response.next_items()  

                continue #no more comments (?)

            # extend output with response comments
            comments.extend(response_json['comments'])
            total_n_comments_read += len(response_json['comments'])
                
            # check if we reached maximum n of requests for this video
            if j >= max_requests:
                break #stop
                
            # get next item
            cursor = response.json().get('cursor')
            response = response.next_items()  

            if response == None:
                break # no more responses
            
            response_json = response.json()
            
            if response_json['comments'] == None:
                print("Continue!")
                response = response.next_items()  

                continue #no more comments (?)
                
        
            j = j + 1

            if (j // 10)==0: print(j)
    # handle exceptions
    except ValidationException as e:
        print(e, e.field)

    except ResponseException as e:
        print(e, e.response.status_code)

    # remove comments from last response object to get the meta data
    response_json['comments'] = None

    # creat dictionary with metadata and comments
    out = {"meta": response_json, "comments": comments}

    return out


def save_comments(file: str, comments: dict):
    with open(file, 'w') as f:
        json.dump(comments, f)


# Download comments

In [120]:
df = df.fillna("")
df["name"] = np.where(df["Politician"]=="", df["Influencer/tiktoker"].replace(" ", "_", regex=True), df["Politician"].replace(" ", "_", regex=True))

In [121]:
df.iloc[4:10]

,Influencer/tiktoker,Politician,Date,Subject,Link,Done,id,name
4,Arianna Biella,,08/09/2022,Personal preferences on proposed issues,https://www.tiktok.com/@ariannabiellas/video/7...,False,7140961766028102918,Arianna_Biella
5,Arianna Biella,,08/09/2022,Personal preferences on proposed issues,https://www.tiktok.com/@ariannabiellas/video/7...,False,7140966608507358470,Arianna_Biella
6,Arianna Biella,,24/09/2022,RDC / work / criminality,https://www.tiktok.com/@ariannabiellas/video/7...,False,7146956596566854918,Arianna_Biella
7,Cotoncri,,20/09/2022,Climate (part 1),https://www.tiktok.com/@cotoncri/video/7145502...,False,7145502591374478597,Cotoncri
8,Cotoncri,,20/09/2022,Climate (part 2),https://www.tiktok.com/@cotoncri/video/7145499...,False,7145499511735995653,Cotoncri
9,Eleonora Agnarelli,,23/08/2022,Electoral program of M5s,https://www.tiktok.com/@eleonora_agnarelli/vid...,False,7135099086641925381,Eleonora_Agnarelli


In [138]:
# iterate over videos
for i, row in df.iloc[16:20].iterrows():
    id = str(row['id'])
    file = _data_path + row['name'] + "_com_" + id + ".json"

    print("\n#########################################\nProcessing video",id)
    print(row["name"])
    comments = get_comments(id)
    save_comments(file, comments)


#########################################
Processing video 7139123631396785413
Berlusconi
2
3
4
5
6
7
8
9
Continue!

#########################################
Processing video 7138834940187151622
Berlusconi
2
3
4
5
6
7
API Key Rate-Limit reached. 429

#########################################
Processing video 7142138056961805573
Berlusconi
API Key Rate-Limit reached. 429


UnboundLocalError: local variable 'response_json' referenced before assignment

# Example for reading data

In [134]:
with open("./data/Partito_Democratico_com_7141021308854258950.json") as infile:
    # returns JSON object as 
    # a dictionary
    data = json.load(infile)


In [135]:
len(data["comments"])

25

In [136]:
for c in data['comments']:
    print(c["text"])

vai Giorgia
Grande Alessandro👏♥️
applausi
bravo Zan è una del 1920 e vuole portarci indietro
Alessandro la vedo dura! Grazie del tuo lavoro!
👏👏👏
Siete belli entrambi 😂😂😂
infatti voto Giorgia
ok ma il pd non ha votato il ddlzan
grande alessandro❤️
forza Giorgia
Resisti Giorgia
forza Giorgia. certo che ci sono le loby
voto Giorgia 🔥
ciao ciao pddini
come sempre fate la campagna elettorale a giorgia meloni... dite cosa proponete non cosa propongono gli altri mi avete fortemente deluso voto conte..
Bla bla bla..
👏👏👏
W giorgia
la più grande FAKE NEWS è il PD dopo tanti anni infiltrati al governo senza mai vincere nulla oggi finalmente il popolo ha deciso
grazie per avermi convinto a non votare Pd
Ma voi siete gli stessi che avete abolito l’articolo 18? 🤔😏👁👁👄👁 non se ne salva uno
Giorgia io là penso come Alessandro e poi h dire l'omosessualità vine insegnata ha scuola elementare ma che dici io etrasesesuale e ho molti amici
I passi da gigante però ci sono stati e negarlo è follia.
partito de

In [129]:
os.listdir(_data_path)

['Cotoncri_com_7145502591374478597.json',
 'video_ids.txt',
 'Arianna_Biella_com_7140961766028102918.json',
 'Arianna Biella_com_7146956596566854918.json',
 'Eleonora_Agnarelli_com_7135099086641925381.json',
 'Arianna_Biella_com_7140954969871600901.json',
 'comment_list.json',
 'Arianna_Biella_com_7146956596566854918.json',
 'Salvini_com_7146104387079900421.json',
 'Meloni_com_7144986246215929094.json',
 'Berlusconi_com_7138834940187151622.json',
 'Cotoncri_com_7145499511735995653.json',
 'Arianna_Biella_com_7140966608507358470.json',
 'video_list.csv',
 'Arianna Biella_com_7140966608507358470.json']

In [113]:
for i, row in df.iloc[5:10].iterrows():
    id = str(row['id'])
    print(_data_path + row['name'] + "_com_" + id + ".json")


./data/Arianna_Biella_com_7140966608507358470.json
./data/Arianna_Biella_com_7146956596566854918.json
./data/Cotoncri_com_7145502591374478597.json
./data/Cotoncri_com_7145499511735995653.json
./data/Eleonora_Agnarelli_com_7135099086641925381.json


<p style="text-align: center;"> <i><b> Fin </b></i> </p>